<img src="https://cybersecurity-excellence-awards.com/wp-content/uploads/2017/06/366812.png">


<h1><center>Darwin Unsupervised Model Building </center></h1>


# Prior to getting started:

Darwin notebook will no longer support 'Register User' starting from 2.0. As a user, you must have credentials ready before using this notebook. 

In order to proceed, in the Environment Variables cell: 
1. Set your username and password to ensure that you're able to log in successfully
2. Set the path to the location of your datasets if you are using your own data.  The path is set for the examples.
  <br><b>NOTE:</b> We provide two ways to analyze feature importance. One is to use the entire dataset; the other one is to analyze a few samples to understand individual samples. In the latter case, we advise users to use a small dataset (<=500) because it takes long time to process individual samples. 

Here are a few things to be mindful of:
1. For every run, check the job status (i.e. requested, failed, running, completed) and wait for job to complete before proceeding. 
2. If you're not satisfied with your model and think that Darwin can benefit from extra training, use the resume function.

## Import Necessary Libraries

In [148]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
import datetime
from IPython.display import Image
from time import sleep
from amb_sdk.sdk import DarwinSdk

## Set Darwin SDK

In [149]:
ds = DarwinSdk()
ds.set_url('https://darwin-api.sparkcognition.com/v1/')

(True, 'https://amb-api.sparkcognition.com/v1/')

## Environment Variables

In [142]:
#Set your user id and password accordingly
USER="[your Darwin user id]"
PW="[your Darwin password]"

# Set path to datasets - The default below assumes Jupyter was started from amb-sdk/examples/Enterprise/
# Modify accordingly if you wish to use your own data
PATH_TO_DATASET = '/Users/ylan/Misc/SpectralNet/Darwin/sets/'
TRAIN_DATASET = 'pulsars.csv'

# PREDICT_DATASET is the data you want to use for feature importance in unsupervised, typically it should be a small
# subset of the TRAIN_DATASET
PREDICT_DATASET = 'pulsars_predict.csv'

# User Login

In [151]:
status, msg = ds.auth_login_user(USER,PW)
if not status:
    print(msg)
else:
    print('You are logged in.')

You are logged in.


# Data Upload

**Read dataset and view a file snippet**

In [152]:
# Preview dataset
df = pd.read_csv(os.path.join(PATH_TO_DATASET, TRAIN_DATASET))
df.head()

,mean_profile,std_profile,kurt_profile,skew_profile,mean_dmsnr,std_dmsnr,kurt_dmsnr,skew_dmsnr,class
0,111.093750,47.341089,0.435469,0.471339,2.386288,15.867173,9.327098,103.545876,0
1,105.000000,49.203341,0.563215,0.382150,1.601171,14.657767,11.381829,148.334350,0
2,115.304688,43.653207,0.448319,0.614359,3.158027,21.378754,8.347430,76.310271,0
3,108.554688,52.559016,0.138068,-0.442340,1.787625,12.108555,11.262459,180.074252,0
4,136.429688,49.552164,-0.180418,0.370338,9.066054,37.284742,4.270014,17.700441,0


**Upload dataset to Darwin**

In [155]:
# Upload dataset
status, dataset = ds.upload_dataset(os.path.join(PATH_TO_DATASET, TRAIN_DATASET))
print(status)
print(dataset)

if not status:
    print(dataset)

False
400: BAD REQUEST - {"message": "Dataset already exists"}

400: BAD REQUEST - {"message": "Dataset already exists"}



# Analyze Data
Before creating a model, users need to analyze data and clean data first. 

In [156]:
ts = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())
status, analyze_id = ds.analyze_data(TRAIN_DATASET, 
                                     job_name = 'Darwin_analyze_data_job' + "-" + ts, 
                                     artifact_name = 'Darwin_analyze_data_artifact' + "-" + ts)
sleep(1)
if status:
    ds.wait_for_job('Darwin_analyze_data_job' + "-" + ts)
else:
    print(analyze_id)

{'status': 'Complete', 'starttime': '2019-07-19T11:31:39.512506', 'endtime': None, 'percent_complete': 100, 'job_type': 'AnalyzeData', 'loss': None, 'generations': None, 'dataset_names': ['pulsars.csv'], 'artifact_names': ['Darwin_analyze_data_artifact-20190719113139'], 'model_name': None, 'job_error': None}


In [157]:
ds.lookup_job_status_name(analyze_id['job_name'])

(True,
 {'status': 'Complete',
  'starttime': '2019-07-19T11:31:39.512506',
  'endtime': None,
  'percent_complete': 100,
  'job_type': 'AnalyzeData',
  'loss': None,
  'generations': None,
  'dataset_names': ['pulsars.csv'],
  'artifact_names': ['Darwin_analyze_data_artifact-20190719113139'],
  'model_name': None,
  'job_error': None})

# Clean Data

Starting Version 1.6, Darwin SDK offers a way to clean your data outside of model training. Every dataset needs to be cleaned before creating a model. There is no need to save the cleaned data and upload it. 

In [158]:
# Clean dataset
status, job_id = ds.clean_data(dataset_name=TRAIN_DATASET)
if not status:
    print(job_id)
else:
    print('Data has been successfully cleaned!')

Data has been successfully cleaned!


# Create and Train Model 

To build unsupervised models, which cluster data and perform anomaly detection, Darwin goes through the following steps:
1. Determines an approximate number of clusters to start with using a single pass with a hierarchical method
2. Iterates on subsets of the data using a Spectral-Net algorithm to determine the ideal number of clusters
3. Proceeds to cluster the data using a Spectral-Net approach

In the cell below, specify the parameters used to create the model:
- model: the name of your model
- max_epochs: the number of epochs to train the model, one epoch indicates one scan of the entire dataset
- n_clusters: the number of clusters, either an integer or 'auto', if left with 'auto', the unsupervised algorithm will compute a number for you

In [159]:
# Build model
ts = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())
model = "model" + "-" + ts
max_epochs = 2
n_clusters = 2
status, job_id = ds.create_model(dataset_names=TRAIN_DATASET,
                                 model_name=model,
                                 max_epochs=max_epochs,
                                 fit_profile_name=job_id['profile_name'],
                                 n_clusters=n_clusters)
sleep(1)
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Running', 'starttime': '2019-07-19T11:31:53.673678', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['pulsars.csv'], 'artifact_names': None, 'model_name': 'model-20190719113152', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-07-19T11:31:53.673678', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['pulsars.csv'], 'artifact_names': None, 'model_name': 'model-20190719113152', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-07-19T11:31:53.673678', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['pulsars.csv'], 'artifact_names': None, 'model_name': 'model-20190719113152', 'job_error': ''}
{'status': 'Complete', 'starttime': '2019-07-19T11:31:53.673678', 'endtime': '2019-07-19T11:32:34.983102', 'percent_complete': 100, 'job_type': 'TrainModel', 'loss': None, 

In [160]:
# look up job status
ds.lookup_job_status_name(job_id['job_name'])

(True,
 {'status': 'Complete',
  'starttime': '2019-07-19T11:31:53.673678',
  'endtime': '2019-07-19T11:32:34.983102',
  'percent_complete': 100,
  'job_type': 'TrainModel',
  'loss': None,
  'generations': 0,
  'dataset_names': ['pulsars.csv'],
  'artifact_names': None,
  'model_name': 'model-20190719113152',
  'job_error': ''})

In [161]:
# look up the model
ds.lookup_model_name(job_id['model_name'])

(True,
 {'id': 'b79ea6a8-aa42-11e9-9217-cb1f117ab849',
  'name': 'model-20190719113152',
  'type': 'Unsupervised',
  'problem_type': None,
  'updated_at': '2019-07-19T11:32:34.965747',
  'trained_on': ['pulsars.csv'],
  'trained_on_id': ['53f603de-a8d6-11e9-bd79-c3084e4b6742'],
  'loss': None,
  'complete': True,
  'generations': 0,
  'parameters': {'impute': 'mean',
   'max_unique_values': 50,
   'max_int_uniques': 15,
   'train_time': '00:10',
   'big_data': False,
   'max_generation': 2,
   'recurrent': None,
   'n_clusters': 2},
  'description': {'genome_type': 'Unsupervised',
   'model': "UnsupervisedPipeline(anomaly=False, anomaly_prior=0.0015, auto_save_per=10,\n           big_data=False, clustering=True,\n           clustermethod='GaussianMixture', file_path=None,\n           job_id='b79f56e8-aa42-11e9-9217-c7234b9e6d0f', max_generation=2,\n           max_time=600,\n           model_file='models/8cf0c5fa-0ded-11e8-93e3-ebbdceea028c_model-20190719113152',\n           n_clusters=

## Extra Training (Optional)
Run the following cell for extra training, no need to specify parameters

In [162]:
# Train some more
extra_epochs = 1
status, job_id = ds.resume_training_model(dataset_names=TRAIN_DATASET,
                                          model_name=model,
                                          max_epochs=extra_epochs,
                                          n_clusters=n_clusters)
sleep(1)
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Running', 'starttime': '2019-07-19T11:34:11.132371', 'endtime': None, 'percent_complete': 0, 'job_type': 'UpdateModel', 'loss': None, 'generations': 0, 'dataset_names': ['pulsars.csv'], 'artifact_names': None, 'model_name': 'model-20190719113152', 'job_error': ''}
{'status': 'Complete', 'starttime': '2019-07-19T11:34:11.132371', 'endtime': '2019-07-19T11:34:18.2635', 'percent_complete': 100, 'job_type': 'UpdateModel', 'loss': None, 'generations': 0, 'dataset_names': ['pulsars.csv'], 'artifact_names': None, 'model_name': 'model-20190719113152', 'job_error': ''}


## Predict
Run the following cell for prediction

In [163]:
# Test model
status, artifact = ds.run_model(TRAIN_DATASET, 
                                model, 
                                supervised=False)
sleep(1)
ds.wait_for_job(artifact['job_name'])

{'status': 'Running', 'starttime': '2019-07-19T11:34:48.016251', 'endtime': None, 'percent_complete': 0, 'job_type': 'RunModel', 'loss': None, 'generations': 0, 'dataset_names': ['pulsars.csv'], 'artifact_names': ['a8226823f3e341578daa0acdd8fff46f'], 'model_name': 'model-20190719113152', 'job_error': ''}
{'status': 'Complete', 'starttime': '2019-07-19T11:34:48.016251', 'endtime': '2019-07-19T11:34:50.537774', 'percent_complete': 100, 'job_type': 'RunModel', 'loss': None, 'generations': 0, 'dataset_names': ['pulsars.csv'], 'artifact_names': ['a8226823f3e341578daa0acdd8fff46f'], 'model_name': 'model-20190719113152', 'job_error': ''}


(True, 'Job completed')

In [164]:
# Get predictions
status, pred_file = ds.download_artifact(artifact['artifact_name'])

In [165]:
# View prediction
df = pd.read_csv(pred_file['filename'])
df.head()

,anomaly_score,predict_proba,prediction
0,-13.880201,"[0.02602722538768635, 0.9739727746123138]",0.430858
1,-13.880201,"[0.010041108295157924, 0.9899588917048421]",0.263699
2,-13.880201,"[0.021460673633545826, 0.9785393263664541]",0.546369
3,-7.874285,"[0.046946763257389265, 0.9530532367426107]",0.361209
4,-7.798066,"[0.8069366623140716, 0.19306333768592857]",1.125853


## Analyze Model
Analyze model provides feature importance ranked by the model. It indicates a general view of which features pose a bigger impact on the model

In [166]:
ts = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())
status, analyze_id = ds.analyze_model(job_id['model_name'], 
                                      job_name='Darwin_analyze_model_job-' + ts, 
                                      artifact_name='Darwin_analyze_model_artifact-' + ts)
sleep(1)
if status:
    ds.wait_for_job('Darwin_analyze_model_job-' + ts)
else:
    print(analyze_id)

{'status': 'Requested', 'starttime': '2019-07-19T11:35:19.378853', 'endtime': None, 'percent_complete': 0, 'job_type': 'AnalyzeModel', 'loss': None, 'generations': 0, 'dataset_names': None, 'artifact_names': ['Darwin_analyze_model_artifact-20190719113518'], 'model_name': 'model-20190719113152', 'job_error': None}
{'status': 'Complete', 'starttime': '2019-07-19T11:35:19.378853', 'endtime': '2019-07-19T11:35:27.986372', 'percent_complete': 100, 'job_type': 'AnalyzeModel', 'loss': None, 'generations': 0, 'dataset_names': None, 'artifact_names': ['Darwin_analyze_model_artifact-20190719113518'], 'model_name': 'model-20190719113152', 'job_error': ''}


In [167]:
ds.lookup_job_status_name('Darwin_analyze_model_job-' + ts)

(True,
 {'status': 'Complete',
  'starttime': '2019-07-19T11:35:19.378853',
  'endtime': '2019-07-19T11:35:27.986372',
  'percent_complete': 100,
  'job_type': 'AnalyzeModel',
  'loss': None,
  'generations': 0,
  'dataset_names': None,
  'artifact_names': ['Darwin_analyze_model_artifact-20190719113518'],
  'model_name': 'model-20190719113152',
  'job_error': ''})

Downloade and print the top 10 features

In [168]:
status, feature_importance = ds.download_artifact('Darwin_analyze_model_artifact-' + ts)
feature_importance

std_profile     0.191714
mean_dmsnr      0.141063
skew_profile    0.111311
kurt_profile    0.107637
kurt_dmsnr      0.106909
mean_profile    0.105243
skew_dmsnr      0.095547
std_dmsnr       0.080367
class           0.060209
dtype: float64

## Analyze Prediction
Different from Analyze Model, the Analyze Prediction provides a way to analyze feature importance for each data point. The output estimates how each feature added or subtracted from a known base-value to result in the overall prediction that was made.  <br>
**You need to set the path to the dataset which contains all the samples you want to analyze (max rows = 500)**

In [169]:
# Upload the data that you are interested in feature importance (max: 500 rows)
status, dataset = ds.upload_dataset(os.path.join(PATH_TO_DATASET, PREDICT_DATASET))
if not status:
    print(dataset)
    
if status:
    dataset_by_row=dataset['dataset_name']
else:
    print("Upload data failed!")

In [174]:
# Clean dataset
status, job_id = ds.clean_data(dataset_name=PREDICT_DATASET, model_name=model)
if not status:
    print(job_id)
else:
    print('Data has been successfully cleaned!')

500: INTERNAL SERVER ERROR - {"message": "Internal Server Error"}



In [124]:
status, analyze_id = ds.analyze_predictions(model, 
                                            PREDICT_DATASET, 
                                            job_name='Analyze_prediction_job-' + ts, 
                                            artifact_name='Analyze_prediction_artifact-' + ts)
sleep(1)
if status:
    ds.wait_for_job('Analyze_prediction_job-' + ts)
else:
    print(analyze_id)

400: BAD REQUEST - {"message": "Clean dataset not found. Please clean dataset before running model."}



In [ ]:
ds.lookup_job_status_name('Analyze_prediction_job-' + ts)

Download and print the top 10 features

In [ ]:
status, feature_importance = ds.download_artifact('Analyze_prediction_artifact-' + ts)
feature_importance.head()